In [357]:
# libraries to use
import requests
import pandas as pd
import json
import io
import itertools
import datetime
import numpy as np

In [2]:
# use these headers for every request to NBA.com
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [450]:
Season='2021-22'
PerMode='Totals'

# make a list of NBA measure types to loop through for a player
MeasureTypes = [
    'Base',
    'Advanced',
    'Misc',
    'Scoring',
    'Usage'
]

# make a list of NBA seasons to loop through later
season_list = [
    '1996-97',
    '1997-98',
    '1998-99',
    '1999-00',
    '2000-01',
    '2001-02',
    '2002-03',
    '2003-04',
    '2004-05',
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2011-12',
    '2012-13',
    '2013-14',
    '2014-15',
    '2015-16',
    '2016-17',
    '2017-18',
    '2018-19',
    '2019-20',
    '2020-21',
    '2021-22',
    '2022-23',
]

In [425]:
# get general (Measure=Base) player total stats (PerMode=Totals) for 2021-22 season (Season=2021-22) - example URL request
# get PLAYER_ID, PLAYER_NAME, GP, MIN
players_stats_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
players_stats_response = requests.get(url=players_stats_url, headers=headers).json()
players_stats_headers = players_stats_response['resultSets'][0]['headers']
players_stats_data = players_stats_response['resultSets'][0]['rowSet']
players_stats_df = pd.DataFrame(data=players_stats_data, columns=players_stats_headers)

In [426]:
player_ids_2021_22 = players_stats_df[['PLAYER_ID', 'PLAYER_NAME','GP','MIN']]
player_ids_2021_22

,PLAYER_ID,PLAYER_NAME,GP,MIN
0,203932,Aaron Gordon,75,2375.418333
1,1630565,Aaron Henry,6,16.983333
2,1628988,Aaron Holiday,63,1020.750000
3,1630174,Aaron Nesmith,52,573.878333
4,1630598,Aaron Wiggins,50,1208.750000
...,...,...,...,...
600,203897,Zach LaVine,67,2328.078333
601,1630285,Zavier Simpson,4,174.083333
602,1630192,Zeke Nnaji,41,697.608333
603,1630533,Ziaire Williams,62,1346.466667


In [429]:
mpg_list = []

for x in range(0, player_ids_2021_22.shape[0]):
    mpg = player_ids_2021_22.iloc[x]['MIN']/player_ids_2021_22.iloc[x]['GP']
    mpg_list.append(round(mpg,1))

player_ids_2021_22.loc[:,'MPG'] = mpg_list

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\553556145.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_ids_2021_22.loc[:,'MPG'] = mpg_list


In [430]:
player_ids_2021_22

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
1,1630565,Aaron Henry,6,16.983333,2.8
2,1628988,Aaron Holiday,63,1020.750000,16.2
3,1630174,Aaron Nesmith,52,573.878333,11.0
4,1630598,Aaron Wiggins,50,1208.750000,24.2
...,...,...,...,...,...
600,203897,Zach LaVine,67,2328.078333,34.7
601,1630285,Zavier Simpson,4,174.083333,43.5
602,1630192,Zeke Nnaji,41,697.608333,17.0
603,1630533,Ziaire Williams,62,1346.466667,21.7


In [443]:
high_mpg_players_2021_22_df = player_ids_2021_22.loc[(player_ids_2021_22['MIN'] >= 500) & (player_ids_2021_22['MPG'] >= 30)]

In [444]:
high_mpg_players_2021_22_df

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
21,203952,Andrew Wiggins,73,2329.513333,31.9
23,203076,Anthony Davis,40,1404.248333,35.1
24,1630162,Anthony Edwards,72,2465.993333,34.2
34,1628389,Bam Adebayo,56,1825.261667,32.6
...,...,...,...,...,...
571,1629639,Tyler Herro,66,2151.011667,32.6
574,1630169,Tyrese Haliburton,77,2694.131667,35.0
575,1630178,Tyrese Maxey,75,2650.356667,35.3
591,203115,Will Barton,71,2276.463333,32.1


In [445]:
high_mpg_players_2021_22_df.to_csv('players_mpg_2021_22_30mpg.csv', index=False)

In [451]:
# example NBA.com 2021-21 (Season=2021-22) players bio (https://stats.nba.com/stats/leaguedashplayerbiostats):
# use for AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT
plyrs_bio_url = 'https://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
plyrs_bio_response = requests.get(url=plyrs_bio_url, headers=headers).json()
plyrs_bio_headers = plyrs_bio_response['resultSets'][0]['headers']
plyrs_bio_data = plyrs_bio_response['resultSets'][0]['rowSet']
plyrs_bio_df = pd.DataFrame(data=plyrs_bio_data, columns=plyrs_bio_headers)
plyrs_bio_headers

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'PLAYER_HEIGHT',
 'PLAYER_HEIGHT_INCHES',
 'PLAYER_WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'GP',
 'PTS',
 'REB',
 'AST',
 'NET_RATING',
 'OREB_PCT',
 'DREB_PCT',
 'USG_PCT',
 'TS_PCT',
 'AST_PCT']

In [452]:
# example NBA.com 2021-21 (Season=2021-22) player index (https://stats.nba.com/stats/playergamelogs/playerindex):
# do we need anything...? Just get from bio?
plyrs_idx_url = 'https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season=2022-23&SeasonType=Regular%20Season='+Season+'&TeamID=0&Weight='
plyrs_idx_response = requests.get(url=plyrs_idx_url, headers=headers).json()
plyrs_idx_headers = plyrs_idx_response['resultSets'][0]['headers']
plyrs_idx_data = plyrs_idx_response['resultSets'][0]['rowSet']
plyrs_idx_df = pd.DataFrame(data=plyrs_idx_data, columns=plyrs_idx_headers)
plyrs_idx_headers

['PERSON_ID',
 'PLAYER_LAST_NAME',
 'PLAYER_FIRST_NAME',
 'PLAYER_SLUG',
 'TEAM_ID',
 'TEAM_SLUG',
 'IS_DEFUNCT',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'JERSEY_NUMBER',
 'POSITION',
 'HEIGHT',
 'WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'ROSTER_STATUS',
 'PTS',
 'REB',
 'AST',
 'STATS_TIMEFRAME',
 'FROM_YEAR',
 'TO_YEAR']

In [453]:
# PlayerID = 2544 # leBron James
# PlayerID = 202695 # kawhi leoard - lots of time off
PlayerID = 1629029 # Luka Doncic

In [456]:
def get_game_logs_df(PlayerID, MeasureType, Season, PerMode):
    glogs_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType='+MeasureType+'&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerID='+str(PlayerID)+'&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
    glogs_response = requests.get(url=glogs_url, headers=headers).json()
    glogs_headers = glogs_response['resultSets'][0]['headers']
    glogs_data = glogs_response['resultSets'][0]['rowSet']
    glogs_df = pd.DataFrame(data=glogs_data, columns=glogs_headers)
    return glogs_df

In [457]:
plyr_base_df = get_game_logs_df(PlayerID,'Base',Season, PerMode)
plyr_advanced_df = get_game_logs_df(PlayerID,'Advanced',Season, PerMode)
plyr_misc_df = get_game_logs_df(PlayerID,'Misc',Season, PerMode)
plyr_scor_df = get_game_logs_df(PlayerID,'Scoring',Season, PerMode)
plyr_usage_df = get_game_logs_df(PlayerID,'Usage',Season, PerMode)

In [465]:
plyr_usage_df.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'USG_PCT', 'PCT_FGM', 'PCT_FGA', 'PCT_FG3M', 'PCT_FG3A',
       'PCT_FTM', 'PCT_FTA', 'PCT_OREB', 'PCT_DREB', 'PCT_REB', 'PCT_AST',
       'PCT_TOV', 'PCT_STL', 'PCT_BLK', 'PCT_BLKA', 'PCT_PF', 'PCT_PFD',
       'PCT_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'USG_PCT_RANK', 'PCT_FGM_RANK', 'PCT_FGA_RANK', 'PCT_FG3M_RANK',
       'PCT_FG3A_RANK', 'PCT_FTM_RANK', 'PCT_FTA_RANK', 'PCT_OREB_RANK',
       'PCT_DREB_RANK', 'PCT_REB_RANK', 'PCT_AST_RANK', 'PCT_TOV_RANK',
       'PCT_STL_RANK', 'PCT_BLK_RANK', 'PCT_BLKA_RANK', 'PCT_PF_RANK',
       'PCT_PFD_RANK', 'PCT_PTS_RANK', 'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

In [466]:
plyr_base_headers_subset = ['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PFD',
 'PTS']

plyr_advanced_headers_subset = ['USG_PCT',
 'PIE',
 'POSS']

plyr_misc_headers_subset= [ 'PTS_PAINT']

plyr_scor_headers_subset = ['PCT_PTS_2PT_MR']

In [467]:
plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset]
plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]

In [470]:
plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                               plyr_advanced_df_subset, 
                               plyr_misc_df_subset, 
                               plyr_scor_df_subset], axis=1)

num_box_stats = plyr_box_stats_df.shape[0]

player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == PlayerID]

# cannot find player birthday info on NBA.com
# subtract age by one when going back single NBA season; this is the work-around for now
player_age = [int(player_bio.iloc[0]['AGE'])] * num_box_stats
plyr_box_stats_df['AGE'] = player_age

player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

In [471]:
fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

fgm_paint_list = []
fgm_mr_list = []
for x in range(0, num_box_stats):
    game_stats = plyr_box_stats_df.iloc[x][fg_attr]
    fgm_paint = game_stats['PTS_PAINT']/2
    fgm_2p = game_stats['FGM'] - game_stats['FG3M']
    fgm_mr = fgm_2p - fgm_paint
    fgm_paint_list.append(int(fgm_paint))
    fgm_mr_list.append(int(fgm_mr))

# note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
# will use 2PT mid-range and in-the-paint made FG for now
plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list


In [472]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,USG_PCT,PIE,POSS,PTS_PAINT,PCT_PTS_2PT_MR,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,FGM_PAINT,FGM_2PT_MR
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,0.368,0.203,61,12,0.000,23,79,230,6,0
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,0.443,0.340,62,10,0.000,23,79,230,5,0
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,0.446,0.134,68,10,0.077,23,79,230,5,1
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,0.405,0.176,80,10,0.063,23,79,230,5,1
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,0.400,0.171,74,8,0.167,23,79,230,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,0.368,0.134,55,4,0.250,23,79,230,2,2
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,0.355,0.103,69,16,0.000,23,79,230,8,0
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,0.353,0.230,75,10,0.308,23,79,230,5,4
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,0.360,0.173,76,14,0.148,23,79,230,7,2


In [473]:
def get_games_past_week(this_game, plyr_box_stats_df):
    # for a given game_date find the week period before
    game_date_str = this_game['GAME_DATE']
    # parse game_date into python date object
    game_date = datetime.datetime.strptime(game_date_str, '%Y-%m-%dT%H:%M:%S')
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago

    # convert game_date to python date objects
    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]

    # create temp df of only game_id and game_date
    game_dates_df = plyr_box_stats_df[['GAME_ID','GAME_DATE']]

    # add converted python game_date object to temp df
    game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = game_dates_df.loc[(game_dates_df['GAME_DATE_OBJ'] <= date_yesterday) & (game_dates_df['GAME_DATE_OBJ'] >= date_week_ago)]
    game_ids_past_week = game_dates_past_week['GAME_ID'].values.tolist()
    games_past_week = plyr_box_stats_df.loc[plyr_box_stats_df['GAME_ID'].isin(game_ids_past_week)]
    
    return games_past_week

In [474]:
'''
Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
Total minutes played - pw_min
Total games played - pw_games
2-point field goal attempts - pw_fg2a
3-point field goal attempts - pw_gf3a
Blocks - pw_blk
Rebounds - pw_reb (need offensive/defensive rebounds…?)
Personal fouls - pw_pf
Assists - pw_ast
'''

'\nCompile box stats containing stat totals from the past week (7 days before) of game_date for the following: \nTotal minutes played - pw_min\nTotal games played - pw_games\n2-point field goal attempts - pw_fg2a\n3-point field goal attempts - pw_gf3a\nBlocks - pw_blk\nRebounds - pw_reb (need offensive/defensive rebounds…?)\nPersonal fouls - pw_pf\nAssists - pw_ast\n'

In [475]:
pw_mins = []
pw_gamess = []
pw_fg3as = []
pw_fg2as = []
pw_orebs = []
pw_drebs = []
pw_asts = []
pw_tovs = []
pw_stls = []
pw_blks = []
pw_posss = []
pw_pfs = []
pw_pfds = []
pw_pts_paints = []
pw_pts_2pt_mrs = []
pw_pie_avg = []
pw_usg_pct_avg = []

In [476]:
# iterate through all games in player box stats and calculate totals/avgs for games the week before
for index, this_game in plyr_box_stats_df.iterrows():
    games_week_before_df = get_games_week_before(this_game, plyr_box_stats_df)
    pw_min = games_week_before_df['MIN'].sum()
    pw_games = games_week_before_df.shape[0]
    pw_fg3a = games_week_before_df['FG3A'].sum()
    pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
    pw_oreb = games_week_before_df['OREB'].sum()
    pw_dreb = games_week_before_df['DREB'].sum()
    pw_ast = games_week_before_df['AST'].sum()
    pw_tov = games_week_before_df['TOV'].sum()
    pw_stl = games_week_before_df['STL'].sum()
    pw_blk = games_week_before_df['BLK'].sum()
    pw_poss = games_week_before_df['POSS'].sum()
    pw_pf = games_week_before_df['PF'].sum()
    pw_pfd = games_week_before_df['PFD'].mean()
    pw_pts_paint = games_week_before_df['PTS_PAINT'].sum()
    pw_pts_2pt_mr = games_week_before_df['PCT_PTS_2PT_MR'].sum()
    pw_pie = games_week_before_df['PIE'].mean()
    pw_usg_pct = games_week_before_df['USG_PCT'].mean()
    
    pw_mins.append(pw_min)
    pw_gamess.append(pw_games)
    pw_fg3as.append(pw_fg3a)
    pw_fg2as.append(pw_fg2a)
    pw_orebs.append(pw_oreb)
    pw_drebs.append(pw_dreb)
    pw_asts.append(pw_ast)
    pw_tovs.append(pw_tov)
    pw_stls.append(pw_stl)
    pw_blks.append(pw_blk)
    pw_posss.append(pw_poss)
    pw_pfs.append(pw_pf)
    pw_pfds.append(pw_pfd)
    pw_pts_paints.append(pw_pts_paint)
    pw_pts_2pt_mrs.append(pw_pts_2pt_mr)
    pw_pie_avg.append(pw_pie)
    pw_usg_pct_avg.append(pw_usg_pct)

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [477]:
plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
plyr_box_stats_df.loc[:,'PW_PTS_PAINT'] = pw_pts_paints
plyr_box_stats_df.loc[:,'PW_PTS_2PT_MR'] = pw_pts_2pt_mrs
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

# fill na with 0 on past week stats
plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

In [478]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_TOV,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,17,4,1,210,6,8.000000,30,0.140,0.216667,0.431333
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,17,4,1,222,9,7.666667,28,0.307,0.160333,0.417000
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,14,4,3,229,6,6.333333,38,0.287,0.192000,0.390333
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,12,2,3,270,8,6.250000,54,0.346,0.220000,0.390000
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,10,3,2,270,5,6.000000,48,0.346,0.229750,0.358500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,16,2,3,220,9,5.333333,40,0.456,0.168667,0.356000
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,14,3,2,228,7,6.333333,28,0.678,0.179667,0.329667
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,10,1,1,153,4,6.000000,18,0.370,0.154500,0.318000
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,5,1,0,77,1,6.000000,4,0.222,0.136000,0.276000


In [479]:
plyr_box_stats_df.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'PTS', 'USG_PCT', 'PIE',
       'POSS', 'PTS_PAINT', 'PCT_PTS_2PT_MR', 'AGE', 'PLAYER_HEIGHT_INCHES',
       'PLAYER_WEIGHT', 'FGM_PAINT', 'FGM_2PT_MR', 'PW_MIN', 'PW_GAMES',
       'PW_FG3A', 'PW_FG2A', 'PW_OREB', 'PW_DREB', 'PW_ASTS', 'PW_TOV',
       'PW_STL', 'PW_BLK', 'PW_POS', 'PW_PF', 'PW_PFD', 'PW_PTS_PAINT',
       'PW_PTS_2PT_MR', 'PW_PIE_AVG', 'PW_USG_PCT_AVG'],
      dtype='object')

In [480]:
plyr_box_stats_df.to_csv('Luka_Doncic_1629029_2021_22_game_logs.csv')

In [481]:
# to-do: work on injury stats

In [482]:
il_data = pd.read_csv('prosportstransactions_scrape_inactivelist_2010_2023.csv')

In [380]:
il_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL,202343.0
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...,201141.0
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...,2038.0
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...,202340.0
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...,2570.0
...,...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL,1630700.0
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL,1628963.0
21441,2/26/2023,Hawks,John Collins (Martin),NaN,activated from IL,NaN
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL,203932.0


In [483]:
mg_data = pd.read_csv('prosportstransactions_scrape_missedgames_2010_2023.csv')

In [484]:
mg_data

,Date,Team,Acquired,Relinquished,Notes
0,8/3/2010,Clippers,NaN,Craig Smith,arthroscopic surgery on right knee (out indefi...
1,8/13/2010,Mavericks,NaN,Rodrigue Beaubois,surgery on left foot to repair broken fifth me...
2,8/14/2010,Warriors,NaN,Ekpe Udoh,surgery on left wrist (out indefinitely)
3,9/21/2010,Raptors,NaN,Ed Davis (a),arthroscopic surgery on right kene to repair t...
4,9/21/2010,Thunder,NaN,Nenad Krstic,surgery on right hand to repair broken finger ...
...,...,...,...,...,...
14440,2/26/2023,Wizards,NaN,Monte Morris,sore lower back (DTD)
14441,2/27/2023,Hornets,NaN,LaMelo Ball,fractured right ankle (out indefinitely)
14442,2/27/2023,Pistons,NaN,Bojan Bogdanovic,sore Achilles (DTD)
14443,2/27/2023,Pistons,NaN,Jaden Ivey,personal reasons (DTD)


In [485]:
player_dict = pd.read_csv('df_nba_player_dict.csv')

In [486]:
player_dict

,Unnamed: 0,isActive,isRookie,namePlayer,idPlayer,countSeasons,yearSeasonFirst,yearSeasonLast,idTeam,hasGamesPlayedFlag,urlPlayerStats,urlPlayerThumbnail,urlPlayerHeadshot,urlPlayerActionPhoto,hasHeadShot,hasThumbnail,hasAction,urlPlayerPhoto
0,1,False,False,Alaa Abdelnaby,76001,4,1990,1994,NaN,False,https://stats.nba.com/player/76001,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76001.png,False,True,True,https://stats.nba.com/media/players/230x185/76...
1,2,False,False,Zaid Abdul-Aziz,76002,9,1968,1977,NaN,False,https://stats.nba.com/player/76002,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76002.png,False,True,True,https://stats.nba.com/media/players/230x185/76...
2,3,False,False,Kareem Abdul-Jabbar,76003,19,1969,1988,NaN,False,https://stats.nba.com/player/76003,https://stats.nba.com/media/players/230x185/76...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/76003.png,True,True,True,https://ak-static.cms.nba.com/wp-content/uploa...
3,4,False,False,Mahmoud Abdul-Rauf,51,10,1990,2000,NaN,False,https://stats.nba.com/player/51,https://stats.nba.com/media/players/230x185/51...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/51.png,False,True,True,https://stats.nba.com/media/players/230x185/51...
4,5,False,False,Tariq Abdul-Wahad,1505,6,1997,2003,NaN,False,https://stats.nba.com/player/1505,https://stats.nba.com/media/players/230x185/15...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/1505.png,False,True,True,https://stats.nba.com/media/players/230x185/15...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,4809,False,False,Ante Zizic,1627790,2,2017,2019,NaN,False,https://stats.nba.com/player/1627790,https://ak-static.cms.nba.com/wp-content/uploa...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/162779...,True,True,False,https://ak-static.cms.nba.com/wp-content/uploa...
4809,4810,False,False,Jim Zoet,78647,0,1982,1982,NaN,False,https://stats.nba.com/player/78647,https://stats.nba.com/media/players/230x185/78...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/78647.png,False,True,True,https://stats.nba.com/media/players/230x185/78...
4810,4811,False,False,Bill Zopf,78648,0,1970,1970,NaN,False,https://stats.nba.com/player/78648,https://stats.nba.com/media/players/230x185/78...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/78648.png,False,True,True,https://stats.nba.com/media/players/230x185/78...
4811,4812,True,False,Ivica Zubac,1627826,6,2016,2022,NaN,False,https://stats.nba.com/player/1627826,https://stats.nba.com/media/players/230x185/16...,https://ak-static.cms.nba.com/wp-content/uploa...,https://stats.nba.com/media/players/700/162782...,True,True,True,https://ak-static.cms.nba.com/wp-content/uploa...


In [487]:
mg_id = []

num_cannot_find = 0

for index, row in mg_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        mg_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        mg_id.append(round(player['idPlayer'].values[0],0))
        
print('cannot find: ' + str(num_cannot_find))

cannot find: Ed Davis (a)
cannot find: Jeff Pendergraph / Jeff Ayres
cannot find: Louis Amundson / Lou Amundson
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Mike Dunleavy Jr.
cannot find: Mike Dunleavy Jr.
cannot find: John Wall (a)
cannot find: Marcus Thornton (T.)
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: J.J. Redick
cannot find: J.J. Redick
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovi

cannot find: Larry Sanders (b)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Thornton (T.)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Malcolm Lee / Malcom Lee
cannot find: Marcus Thornton (T.)
cannot find: Louis Williams / Lou Williams
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Louis Williams / Lou Williams
cannot find: J.J. Redick
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: James Jones (b)
cannot find: John Wall (a)
cannot find: Chris Smith (c)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Maurice Harkless / Moe Harkless
cannot find: Gerald Henderson (b)
cannot find: Maurice Harkless / Moe Harkless
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Maurice Williams / Mo Will

cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Ed Davis (a)
cannot find: Greg Smith (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike James (Lamont)
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Mike Conley Jr.
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike Conley Jr.
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Metta World Peace / Ron Artest
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Greg Smith (a)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smi

cannot find: Mike Conley Jr.
cannot find: J.J. Redick
cannot find: Maurice Williams / Mo Williams
cannot find: Maurice Williams / Mo Williams
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: J.J. Redick
cannot find: Mike Conley Jr.
cannot find: Maurice Williams / Mo Williams
cannot find: Maurice Williams / Mo Williams
cannot find: J.J. Redick
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: J.J. Redick
cannot find: James Ennis
cannot find: Larry Sanders (b)
cannot find: James Ennis
cannot find: Larry Sanders (b)
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Louis Williams / Lou Williams
cannot find: Enes Kanter
cannot find: Enes Kanter
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Louis Amundson / Lou Amundson
cannot find: Louis Amundson / Lou Amundson
cannot find: Mike Conley Jr.
cannot

cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Maurice Harkless / Moe Harkless
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Harkless / Moe Harkless
cannot find: Wesley Johnson / Wes Johnson
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Ed Davis (a)
cannot find: Patrick Mills / Patty Mills
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Wesley Johnson / Wes Johnson
cannot find: (William) Tony Parker
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Gerald Green (b)
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: D.J. Augustine
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Ed Davis (a)
cannot find: Maurice Harkless / Moe Harkless
cannot find: Enes Kanter
cannot find: D.J. Augustine
cannot find: En

cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: J.R. Smith
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: (William) Tony Parker
cannot find: (William) Tony Parker
cannot find: Gerald Green (b)
cannot find: Zhou Qi / Qi Zhou
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Gerald Green (b)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Louis Williams / Lou Williams
cannot find: John Wall (a)
cannot find: Zhou Qi / Qi Zhou
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: John Wall (a)
cannot find: D.J. Augusti

cannot find: Marcus Morris
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Marcus Morris
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Marcus Morris
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: (Sean) Chris Smith
cannot find: Cameron Thomas / Cam Thomas
cannot find: Cameron Thomas / Cam Thomas
cannot find: Louis Williams / Lou Williams
cannot find: (Sean) Chris Smith
cannot find: Louis Williams / Lou Williams
cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Ja'Vonte Smart / Javonte Smart
cannot fin

In [489]:
len(mg_id)

14445

In [490]:
mg_data.loc[:,'PLAYER_ID'] = mg_id

In [492]:
il_id = []

num_cannot_find = 0

for index, row in il_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        il_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        il_id.append(round(player['idPlayer'].values[0],0))
        
print('cannot find: ' + str(num_cannot_find))

cannot find: Matt Janning
cannot find: Maurice Williams / Mo Williams
cannot find: Alexis Ajinca / Alex Ajinca
cannot find: A.J. Price
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Louis Amundson / Lou Amundson
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Williams / Mo Williams
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Patrick Mills / Patty Mills
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Maurice Williams / Mo Williams
cannot find: Marcus Thornton (T.)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Gerald 

cannot find: Ishmael Smith / Ish Smith
cannot find: Patrick Mills / Patty Mills
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Greg Smith (a)
cannot find: Patrick Mills / Patty Mills
cannot find: Malcolm Thomas (b)
cannot find: Bill Walker
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Chris Johnson (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Chris Johnson (a)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: C.J. Miles
cannot find: (William) Tony Parker
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Walker Russell Jr.
cannot find: Courtney Simpson
cannot find

cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: James Jones (b)
cannot find: James Jones (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: J.J. Redick
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: Tony Mitchell (b)
cannot find: A.J. Price
cannot find: Ray McCallum Jr.
cannot find: Malcolm Thomas (b)
cannot find: Ray McCallum Jr.
cannot find: Byron Mullens / B.J. Mullens
cannot find: Metta World Peace / Ron Artest
cannot find: A.J. Price
cannot find: James Jones (b)
cannot find: Byron Mullens / B.J. Mullens
cannot find: A.J. Price
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: A.J. Price
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike Conley

cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: John Wall (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: Mike Conley Jr.
cannot find: Greg Smith (a)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Mike Dunleavy Jr.
cannot find: Walter Tavares / Edy Tavares
cannot find: Norris Cole (a)
cannot find: Tony Wroten Jr.
cannot find: Gerald Henderson (b)
cannot find: R.J. Hunter
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Louis Amundson / Lou Amundson
cannot find: Metta World Peace / Ron Artest
cannot find: Ro

cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Metta World Peace / Ron Artest
cannot find: Wesley Johnson / Wes Johnson
cannot find: C.J. Miles
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Metta World Peace / Ron Artest
cannot find: Stephen Zimmerman Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Kahlil Felder / Kay Felder
cannot find: Stephen Zimmerman Jr.
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Stephen Zimmerman Jr.
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Mike Dunleavy Jr.
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: (William) Tony Parker
cannot find: Emanuel Ginobili / Manu Ginob

cannot find: Norris Cole (a)
cannot find: James Ennis
cannot find: (William) Tony Parker
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Marcus Morris
cannot find: Wayne Selden Jr.
cannot find: Alexis Ajinca / Alex Ajinca
cannot find: (William) Tony Parker
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Vincent Hunter / Vince Hunter
cannot find: A.J. Hammons
cannot find: Eric Griffin
cannot find: Raulzinho Neto / Raul Neto
cannot find: Harry Giles
cannot find: Wesley Iwundu / Wes Iwundu
cannot find: Jacob Wiley / Jake Wiley
cannot find: Yakuba Ouattara / Billy Ouattara
cannot find: Mike Young
cannot find: Sheldon McClellan / Sheldon Mac
cannot find: K.J. McDaniels
cannot find: P.J. Dozier
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Raulzinho Neto / Raul Neto
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find

cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Wayne Selden Jr.
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: R.J. Hunter
cannot find: Zhou Qi / Qi Zhou
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: John Wall (a)
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Marcus Morris
cannot find: Raulzinho Neto / Raul Neto
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: Ed Davis (a)
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Marcus Morris
cannot find: R.J. Hunter
cannot find: John Wall (a)
cannot find: Frank Mason
cannot find: Kahlil Felder / Kay Felder
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: R.J. Hunter
cannot find: John Wall (a)
cannot find: Alexis Ajinca / Alex Ajinca
cannot find: Maurice H

cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Morris
cannot find: Maurice Harkless / Moe Harkless
cannot find: Mike Conley Jr.
cannot find: Maurice Harkless / Moe Harkless
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Enes Kanter / Enes Freedom
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Mike Conley Jr.
cannot find: Mike Conley Jr.
cannot find: Moritz Wagner / Moe Wagner
cannot find: Enes Kanter / Enes Freedom
cannot find: Cameron Reddish / Cam Reddish
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Ed Davis (Adam)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Maurice Harkless / Moe Harkless
cannot find: Cameron Reddish / Cam Reddish
cannot find: K.Z. Okpala
cannot find: Mike Conley Jr.
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Ed Davis (Adam)
cannot fi

cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Ed Davis (Adam)
cannot find: Daulton Hommes
cannot find: Louis Williams / Lou Williams
cannot find: Enes Freedom / Enes Kanter
cannot find: Cameron Thomas / Cam Thomas
cannot find: J.T. Thor
cannot find: C.J. Miles
cannot find: Justin Jackson (Aaron)
cannot find: Wesley Iwundu / Wes Iwundu
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: B.J. Johnson
cannot find: D.J. Augustine
cannot find: Raulzinho Neto / Raul Neto
cannot find: Marcus Morris
cannot find: J.T. Thor
cannot find: Ignas Brazdeikis / Iggy Brazdeikis
cannot find: Moritz Wagner / Moe Wagner
cannot find: Kenyon Martin Jr. / K.J. Martin
cannot find: B.J. Johnson
cannot find: Malik Ellison
cannot find: Wayne Selden Jr.
cannot find: Justin Jackson (Aaron)
cannot find: Brandon Boston Jr. / B.J. Boston Jr.
cannot find: W

In [493]:
il_data.loc[:,'PLAYER_ID'] = il_id

In [494]:
len(il_id)


21444

In [495]:
il_data.loc[:,'PLAYER_ID'] = il_id


In [498]:
il_data

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL,202343.0
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...,201141.0
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...,2038.0
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...,202340.0
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...,2570.0
...,...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL,1630700.0
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL,1628963.0
21441,2/26/2023,Hawks,John Collins,NaN,activated from IL,1628381.0
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL,203932.0


In [497]:

#player = player_dict.loc[player_dict['namePlayer'] == 'LeBron James']
player = player_dict.loc[player_dict['namePlayer'] == 'Luka Doncic']
player['idPlayer'].values[0]

1629029

In [499]:
player_il = il_data.loc[il_data['PLAYER_ID'] == 1629029]
player_mg = mg_data.loc[mg_data['PLAYER_ID'] == 1629029]


In [500]:
player_il

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
12875,12/2/2018,Mavericks,NaN,Luka Doncic,placed on IL with strained right hip,1629029.0
12885,12/4/2018,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0
13607,2/22/2019,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,1629029.0
13640,2/25/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0
13813,3/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right knee,1629029.0
13839,3/18/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0
13955,3/31/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0
13987,4/3/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0
14004,4/5/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,1629029.0
14043,4/9/2019,Mavericks,Luka Doncic,NaN,activated from IL,1629029.0


In [501]:
player_mg

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
11133,1/31/2019,Mavericks,NaN,Luka Doncic,sore left ankle (DTD),1629029.0
11148,2/2/2019,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0
11409,3/30/2019,Mavericks,NaN,Luka Doncic,bruised right thigh (DTD),1629029.0
11492,4/10/2019,Mavericks,NaN,Luka Doncic,left thigh injury (out for season),1629029.0
11831,12/15/2019,Mavericks,NaN,Luka Doncic,sprained right ankle (DTD),1629029.0
12266,8/9/2020,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0
12276,8/11/2020,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0
12405,1/3/2021,Mavericks,NaN,Luka Doncic,bruised left quadricep (DTD),1629029.0
12412,1/4/2021,Mavericks,Luka Doncic,NaN,returned to lineup,1629029.0
12799,3/11/2021,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),1629029.0


In [406]:
schedule_df = pd.read_csv("schedule_2022_23.csv")

In [407]:
schedule_df

,Unnamed: 0,typeSeason,slugSeason,idGame,dateGame,slugMatchup,slugTeamWinner,slugTeamLoser,numberGameDay
0,1,Regular Season,2022-23,22200001,2022-10-18,BOS vs. PHI,BOS,PHI,1
1,2,Regular Season,2022-23,22200002,2022-10-18,LAL @ GSW,GSW,LAL,2
2,3,Regular Season,2022-23,22200003,2022-10-19,ORL @ DET,DET,ORL,1
3,4,Regular Season,2022-23,22200004,2022-10-19,WAS @ IND,WAS,IND,2
4,5,Regular Season,2022-23,22200005,2022-10-19,HOU @ ATL,ATL,HOU,3
...,...,...,...,...,...,...,...,...,...
931,932,Regular Season,2022-23,22200933,2023-03-01,PHX @ CHA,CHA,CHA,1
932,933,Regular Season,2022-23,22200934,2023-03-01,CHI @ DET,DET,DET,2
933,934,Regular Season,2022-23,22200935,2023-03-01,BOS vs. CLE,CLE,CLE,3
934,935,Regular Season,2022-23,22200936,2023-03-01,PHI @ MIA,MIA,MIA,4


In [414]:
sch_game_dates = schedule_df['dateGame'].values.tolist()
sch_game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in sch_game_dates]
schedule_df.loc[:,'dateGame_OBJ']  = sch_game_dates_obj

In [416]:
team_abbrev = schedule_df.slugTeamWinner.unique()
team_abbrev

array(['BOS', 'GSW', 'DET', 'WAS', 'ATL', 'NOP', 'CHI', 'TOR', 'MEM',
       'MIN', 'CHA', 'UTA', 'PHX', 'POR', 'MIL', 'LAC', 'SAS', 'BKN',
       'NYK', 'DEN', 'IND', 'MIA', 'CLE', 'DAL', 'PHI', 'HOU', 'OKC',
       'ORL', 'SAC', 'LAL'], dtype=object)

In [421]:

team_dates_dict = {}

for team in team_abbrev:
    
    team_games = schedule_df.loc[(schedule_df['slugTeamWinner'] == team) | (schedule_df['slugTeamLoser'] == team)]
    team_dates_dict[team]=team_games
    


In [422]:
team_dates_dict

{'BOS':      Unnamed: 0      typeSeason slugSeason    idGame    dateGame  slugMatchup  \
 0             1  Regular Season    2022-23  22200001  2022-10-18  BOS vs. PHI   
 21           22  Regular Season    2022-23  22200022  2022-10-21    BOS @ MIA   
 29           30  Regular Season    2022-23  22200030  2022-10-22    BOS @ ORL   
 46           47  Regular Season    2022-23  22200047  2022-10-24  CHI vs. BOS   
 71           72  Regular Season    2022-23  22200072  2022-10-28  BOS vs. CLE   
 ..          ...             ...        ...       ...         ...          ...   
 867         868  Regular Season    2022-23  22200869  2023-02-14    BOS @ MIL   
 875         876  Regular Season    2022-23  22200877  2023-02-15  BOS vs. DET   
 885         886  Regular Season    2022-23  22200887  2023-02-23    BOS @ IND   
 906         907  Regular Season    2022-23  22200908  2023-02-25  PHI vs. BOS   
 919         920  Regular Season    2022-23  22200921  2023-02-27  NYK vs. BOS   
 
     sl

In [410]:





for idx, row in schedule_df.iterrows():
    game_date_str = row['dateGame']
    game_date = datetime.datetime.strptime(game_date_str, '%Y-%m-%d')
    game_date_objs.append()
    
    print(game_date)
    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]

    # create temp df of only game_id and game_date
    game_dates_df = plyr_box_stats_df[['GAME_ID','GAME_DATE']]

    # add converted python game_date object to temp df
    game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj



2022-10-18 00:00:00
2022-10-18 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-19 00:00:00
2022-10-20 00:00:00
2022-10-20 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-21 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-22 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-23 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00
2022-10-24 00:00:00


In [ ]:
def get_games_past_week(this_game, plyr_box_stats_df):
    # for a given game_date find the week period before
    game_date_str = this_game['GAME_DATE']
    # parse game_date into python date object
    game_date = datetime.datetime.strptime(game_date_str, '%Y-%m-%dT%H:%M:%S')
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago

    # convert game_date to python date objects
    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]

    # create temp df of only game_id and game_date
    game_dates_df = plyr_box_stats_df[['GAME_ID','GAME_DATE']]

    # add converted python game_date object to temp df
    game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = game_dates_df.loc[(game_dates_df['GAME_DATE_OBJ'] <= date_yesterday) & (game_dates_df['GAME_DATE_OBJ'] >= date_week_ago)]
    game_ids_past_week = game_dates_past_week['GAME_ID'].values.tolist()
    games_past_week = plyr_box_stats_df.loc[plyr_box_stats_df['GAME_ID'].isin(game_ids_past_week)]
    
    return games_past_week

In [408]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_TOV,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG
0,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200913,2023-02-26T00:00:00,DAL vs. LAL,L,...,1,2,2,65,0,7.000000,14,0.000,0.264000,0.319000
1,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200891,2023-02-23T00:00:00,DAL vs. SAS,W,...,0,0,0,0,0,0.000000,0,0.000,0.000000,0.000000
2,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200881,2023-02-15T00:00:00,DAL @ DEN,L,...,10,1,0,158,3,7.000000,32,0.074,0.161500,0.341000
3,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200865,2023-02-13T00:00:00,DAL vs. MIN,L,...,6,0,0,85,0,4.000000,16,0.074,0.121000,0.304000
4,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200854,2023-02-11T00:00:00,DAL @ SAC,L,...,0,0,0,0,0,0.000000,0,0.000,0.000000,0.000000
5,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200782,2023-02-02T00:00:00,DAL vs. NOP,W,...,4,2,0,81,2,7.000000,20,0.075,0.177500,0.330500
6,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200761,2023-01-30T00:00:00,DAL vs. DET,W,...,3,1,0,88,3,4.500000,26,0.146,0.097500,0.320500
7,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200734,2023-01-26T00:00:00,DAL @ PHX,W,...,10,3,0,220,6,8.666667,48,0.274,0.231333,0.389667
8,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200717,2023-01-24T00:00:00,DAL vs. WAS,L,...,14,2,0,222,6,8.000000,34,0.195,0.187000,0.358667
9,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200701,2023-01-22T00:00:00,DAL vs. LAC,L,...,10,1,0,152,3,7.500000,24,0.126,0.201500,0.346500
